In [1]:
#trial creating ngram tree


In [2]:
library(dplyr)
library(tidytext)
library(data.tree)
library(stringr)



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [35]:
subsampledText <- readRDS(file='../data/subsets/enUsTwitter1kSubset.rds')


In [36]:
head(subsampledText)

[[1]]
[1] "A Monthly payment plan is now available for those wishing to enroll in Hawaii's #1 RE Investor training program - the Millionaire Mentor..."

[[2]]
[1] "Game time!"

[[3]]
[1] "“: I'm listening to the wacky sounds of - where the Beatles meet the Addams family... and I'm Wednesday” Thx!"

[[4]]
[1] "After watching this Shane Mosley fight from last Saturday, Shane needs to go ahead and retire. He's a hall of famer. Now hang it up..."

[[5]]
[1] "My head hurt"

[[6]]
[1] "Let's all say something nice to everyone we pass today."

In [37]:
#copy/paste processing from
# https://www.tidytextmining.com/tidytext.html

sosToken <- 'start-of-sequence' #'root' token, representing start of a sequence
eolToken <- 'eol' #token to capture end of line

text <- unlist(subsampledText)

text_df <- tibble(line = 1:length(text), text = text)
text_df <- mutate(text_df, text = paste(text, eolToken))

words <- text_df %>%
  unnest_tokens(word, text) %>%
  select(word)

words$index <- 1:nrow(words)

head(text_df)

head(words, 20)
tail(words)

line,text
<int>,<chr>
1,A Monthly payment plan is now available for those wishing to enroll in Hawaii's #1 RE Investor training program - the Millionaire Mentor... eol
2,Game time! eol
3,“: I'm listening to the wacky sounds of - where the Beatles meet the Addams family... and I'm Wednesday” Thx! eol
4,"After watching this Shane Mosley fight from last Saturday, Shane needs to go ahead and retire. He's a hall of famer. Now hang it up... eol"
5,My head hurt eol
6,Let's all say something nice to everyone we pass today. eol


word,index
<chr>,<int>
a,1
monthly,2
payment,3
plan,4
is,5
now,6
available,7
for,8
those,9


word,index
<chr>,<int>
make,13902
the,13903
most,13904
of,13905
it,13906
eol,13907


In [57]:
expandNode <- function(node) {
    
    #table to record newly created nodes from the expansion:
    newNodes <- tibble(nodePath=character(), nOccurrences=integer())
    
    if (node$name == eolToken){
        print("EOL token: will not expand")
        return(newNodes)
    }
    
    nextWordIndices <- node$occurrences + 1
    nextWords <- words[nextWordIndices,]
    
    
    for (uniqueWord in unique(nextWords$word)){
        suppressWarnings(
            newNode <- node$AddChild(uniqueWord)
        )
        newNode$occurrences <- filter(nextWords, word==uniqueWord)$index
        newNode$nOccurrences <- length(newNode$occurrences)
        nodePath <- newNode$pathString
        nodePath <- str_replace(nodePath, sosToken, '.')
        newNodes <- bind_rows(
            tibble(nodePath=nodePath, nOccurrences=newNode$nOccurrences),
            newNodes
        )
    }
    if (nrow(newNodes) > 0)
    {
        print("Created new nodes:")
        newNodes <- arrange(newNodes, desc(nOccurrences))
        print(newNodes)
        Sort(node, 'nOccurrences', decreasing = T)
    }
    newNodes
}

In [58]:
expandFrontier <- function(frontier, rootNode) {
    #expand next node:
    # take the next most common sequence in the frontier
    pathToExpand <- frontier$nodePath[1]
    print(paste('Expanding:', pathToExpand))
    nodeToExpand <- Navigate(rootNode, pathToExpand)

    # remove it from the frontier
    frontier <- frontier[-1,]

    # expand it
    newNodes <- expandNode(nodeToExpand)

    # add new nodes to frontier
    frontier <- bind_rows(frontier, newNodes)
    frontier <- arrange(frontier, desc(nOccurrences))
    frontier
}    
    

In [65]:
rootNode <- Node$new(sosToken)
rootNode$occurrences <- 0:(nrow(words)-1)
rootNode$nOccurrences <- length(rootNode$occurrences)
frontier <-tibble(nodePath='.', nOccurrences=rootNode$nOccurrences)
frontier

nodePath,nOccurrences
<chr>,<int>
.,13907


In [66]:
while (frontier$nOccurrences[1] > 1){
    frontier <- expandFrontier(frontier, rootNode)
    print('Current frontier:')
    print(frontier)
    flush.console()
}

[1] "Expanding: ."
[1] "Created new nodes:"
# A tibble: 3,532 x 2
   nodePath nOccurrences
   <chr>           <int>
 1 ./eol            1000
 2 ./the             374
 3 ./to              361
 4 ./i               325
 5 ./you             259
 6 ./a               257
 7 ./for             178
 8 ./and             176
 9 ./in              159
10 ./of              153
# … with 3,522 more rows
[1] "Current frontier:"
# A tibble: 3,532 x 2
   nodePath nOccurrences
   <chr>           <int>
 1 ./eol            1000
 2 ./the             374
 3 ./to              361
 4 ./i               325
 5 ./you             259
 6 ./a               257
 7 ./for             178
 8 ./and             176
 9 ./in              159
10 ./of              153
# … with 3,522 more rows
[1] "Expanding: ./eol"
[1] "EOL token: will not expand"
[1] "Current frontier:"
# A tibble: 3,531 x 2
   nodePath nOccurrences
   <chr>           <int>
 1 ./the             374
 2 ./to              361
 3 ./i               325
 4 ./you   

[1] "Expanding: ./for"
[1] "Created new nodes:"
# A tibble: 101 x 2
   nodePath        nOccurrences
   <chr>                  <int>
 1 ./for/the                 33
 2 ./for/a                   12
 3 ./for/me                  10
 4 ./for/this                 4
 5 ./for/some                 4
 6 ./for/following            4
 7 ./for/your                 3
 8 ./for/all                  3
 9 ./for/that                 3
10 ./for/being                3
# … with 91 more rows
[1] "Current frontier:"
# A tibble: 4,591 x 2
   nodePath nOccurrences
   <chr>           <int>
 1 ./and             176
 2 ./in              159
 3 ./of              153
 4 ./is              152
 5 ./it              134
 6 ./on              125
 7 ./my              123
 8 ./that            113
 9 ./me              102
10 ./this             84
# … with 4,581 more rows
[1] "Expanding: ./and"
[1] "Created new nodes:"
# A tibble: 129 x 2
   nodePath     nOccurrences
   <chr>               <int>
 1 ./and/i                13


[1] "Expanding: ./my"
[1] "Created new nodes:"
# A tibble: 101 x 2
   nodePath     nOccurrences
   <chr>               <int>
 1 ./my/life               9
 2 ./my/mom                4
 3 ./my/own                3
 4 ./my/kids               2
 5 ./my/age                2
 6 ./my/wife               2
 7 ./my/friends            2
 8 ./my/first              2
 9 ./my/phone              2
10 ./my/next               2
# … with 91 more rows
[1] "Current frontier:"
# A tibble: 5,272 x 2
   nodePath nOccurrences
   <chr>           <int>
 1 ./that            113
 2 ./me              102
 3 ./this             84
 4 ./your             82
 5 ./be               81
 6 ./at               78
 7 ./are              76
 8 ./with             76
 9 ./have             75
10 ./not              65
# … with 5,262 more rows
[1] "Expanding: ./that"
[1] "Created new nodes:"
# A tibble: 93 x 2
   nodePath     nOccurrences
   <chr>               <int>
 1 ./that/eol              6
 2 ./that/you              4
 3 ./tha

[1] "Expanding: ./are"
[1] "Created new nodes:"
# A tibble: 50 x 2
   nodePath      nOccurrences
   <chr>                <int>
 1 ./are/you               10
 2 ./are/the                5
 3 ./are/doing              4
 4 ./are/both               2
 5 ./are/all                2
 6 ./are/not                2
 7 ./are/amazing            2
 8 ./are/too                2
 9 ./are/being              2
10 ./are/on                 2
# … with 40 more rows
[1] "Current frontier:"
# A tibble: 5,714 x 2
   nodePath nOccurrences
   <chr>           <int>
 1 ./with             76
 2 ./have             75
 3 ./not              65
 4 ./but              61
 5 ./we               61
 6 ./just             59
 7 ./like             59
 8 ./so               58
 9 ./do               56
10 ./all              54
# … with 5,704 more rows
[1] "Expanding: ./with"
[1] "Created new nodes:"
# A tibble: 44 x 2
   nodePath    nOccurrences
   <chr>              <int>
 1 ./with/you             7
 2 ./with/my              6


[1] "Expanding: ./like"
[1] "Created new nodes:"
# A tibble: 38 x 2
   nodePath        nOccurrences
   <chr>                  <int>
 1 ./like/a                   8
 2 ./like/to                  6
 3 ./like/the                 4
 4 ./like/that                3
 5 ./like/i                   3
 6 ./like/this                2
 7 ./like/me                  2
 8 ./like/there               1
 9 ./like/tf                  1
10 ./like/marriage            1
# … with 28 more rows
[1] "Current frontier:"
# A tibble: 5,998 x 2
   nodePath nOccurrences
   <chr>           <int>
 1 ./so               58
 2 ./do               56
 3 ./all              54
 4 ./out              53
 5 ./up               53
 6 ./what             50
 7 ./from             50
 8 ./love             49
 9 ./good             49
10 ./u                45
# … with 5,988 more rows
[1] "Expanding: ./so"
[1] "Created new nodes:"
# A tibble: 43 x 2
   nodePath    nOccurrences
   <chr>              <int>
 1 ./so/much              6
 2 ./

...

[1] "Expanding: ./here/we/go"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath          nOccurrences
  <chr>                    <int>
1 ./here/we/go/seas            1
2 ./here/we/go/eol             1
[1] "Current frontier:"
# A tibble: 12,349 x 2
   nodePath                 nOccurrences
   <chr>                           <int>
 1 ./im/going/to                       2
 2 ./stop/listening/to                 2
 3 ./why/do/i                          2
 4 ./something/you've/never            2
 5 ./have/to/do/something              2
 6 ./week/looking/forward              2
 7 ./thank/you/so/much                 2
 8 ./take/me/with                      2
 9 ./in/a/long/time                    2
10 ./if/i/were/to                      2
# … with 12,339 more rows
[1] "Expanding: ./im/going/to"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath          nOccurrences
  <chr>                    <int>
1 ./im/going/to/be             1
2 ./im/going/to/bed            1
[1] "Current frontier:"


[1] "Expanding: ./take/me/with"
[1] "Created new nodes:"
# A tibble: 1 x 2
  nodePath           nOccurrences
  <chr>                     <int>
1 ./take/me/with/you            2
[1] "Current frontier:"
# A tibble: 12,354 x 2
   nodePath              nOccurrences
   <chr>                        <int>
 1 ./in/a/long/time                 2
 2 ./if/i/were/to                   2
 3 ./maybe/one/day                  2
 4 ./coming/soon/eol                2
 5 ./it's/not/your/fault            2
 6 ./trying/to/be/good              2
 7 ./i/wish/i/could                 2
 8 ./hope/you/have/a                2
 9 ./cant/wait/to                   2
10 ./i/would/walk/500               2
# … with 12,344 more rows
[1] "Expanding: ./in/a/long/time"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath              nOccurrences
  <chr>                        <int>
1 ./in/a/long/time/that            1
2 ./in/a/long/time/i               1
[1] "Current frontier:"
# A tibble: 12,355 x 2
   nodePath           

[1] "Expanding: ./hope/you/have/a"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath                   nOccurrences
  <chr>                             <int>
1 ./hope/you/have/a/nice                1
2 ./hope/you/have/a/creative            1
[1] "Current frontier:"
# A tibble: 12,360 x 2
   nodePath                    nOccurrences
   <chr>                              <int>
 1 ./cant/wait/to                         2
 2 ./i/would/walk/500                     2
 3 ./role/model/and                       2
 4 ./us/know/what/you                     2
 5 ./next/week/looking/forward            2
 6 ./what's/my/age/again                  2
 7 ./let/us/know/what/you                 2
 8 ./the/thing/you/do                     2
 9 ./and/it's/not/even                    2
10 ./is/now/available/for                 2
# … with 12,350 more rows
[1] "Expanding: ./cant/wait/to"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath             nOccurrences
  <chr>                       <int>
1 ./ca

[1] "Expanding: ./let/us/know/what/you"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath                     nOccurrences
  <chr>                               <int>
1 ./let/us/know/what/you/find             1
2 ./let/us/know/what/you/think            1
[1] "Current frontier:"
# A tibble: 12,366 x 2
   nodePath                       nOccurrences
   <chr>                                 <int>
 1 ./the/thing/you/do                        2
 2 ./and/it's/not/even                       2
 3 ./is/now/available/for                    2
 4 ./week/looking/forward/to                 2
 5 ./thank/you/so/much/for                   2
 6 ./take/me/with/you                        2
 7 ./next/week/looking/forward/to            2
 8 ./stock                                   1
 9 ./gb                                      1
10 ./gingrich                                1
# … with 12,356 more rows
[1] "Expanding: ./the/thing/you/do"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath               

[1] "Expanding: ./next/week/looking/forward/to"
[1] "Created new nodes:"
# A tibble: 2 x 2
  nodePath                               nOccurrences
  <chr>                                         <int>
1 ./next/week/looking/forward/to/meeting            1
2 ./next/week/looking/forward/to/dinner             1
[1] "Current frontier:"
# A tibble: 12,373 x 2
   nodePath     nOccurrences
   <chr>               <int>
 1 ./stock                 1
 2 ./gb                    1
 3 ./gingrich              1
 4 ./mt                    1
 5 ./eyeroll               1
 6 ./distracted            1
 7 ./internet              1
 8 ./x.x                   1
 9 ./dana                  1
10 ./corner                1
# … with 12,363 more rows


In [63]:
rootNode$

[1] 153

In [10]:
print(rootNode, 'nOccurrences', limit = 20)

Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'super' not found”Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'self' not found”Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'private' not found”

                      levelName nOccurrences
1  start-of-sequence                  137274
2   ¦--eol                             10000
3   ¦--the                              3915
4   ¦--to                               3377
5   ¦--i                                3024
6   ¦--a                                2548
7   ¦--you                              2367
8   ¦--and                              1834
9   ¦--for                              1662
10  ¦--is                               1571
11  ¦--in                               1548
12  ¦--of                               1525
13  ¦--it                               1244
14  ¦--my                               1231
15  ¦--on                               1195
16  ¦--that                             1019
17  ¦--me                                832
18  ¦--at                                767
19  ¦--be                                754
20  °--... 15482 nodes w/ 0 sub           NA


In [11]:
expandNode(rootNode$the)

Warning message in CheckNameReservedWord(name, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'path' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'path2' instead.”Warning message in CheckNameReservedWord(name, check):
“Name 'level' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'level2' instead.”

# A tibble: 1,956 x 2
   nodePath           nOccurrences
   <chr>                     <int>
 1 ./the/coconut                 1
 2 ./the/control                 1
 3 ./the/championship            1
 4 ./the/blazers                 1
 5 ./the/nygiants                1
 6 ./the/boom                    1
 7 ./the/westside                1
 8 ./the/grammys                 1
 9 ./the/paths                   1
10 ./the/corner                  1
# … with 1,946 more rows


nodePath,nOccurrences
<chr>,<int>
./the/coconut,1
./the/control,1
./the/championship,1
./the/blazers,1
./the/nygiants,1
./the/boom,1
./the/westside,1
./the/grammys,1
./the/paths,1


In [12]:
expandNode(rootNode$'the'$'best')

# A tibble: 65 x 2
   nodePath                nOccurrences
   <chr>                          <int>
 1 ./the/best/stickum                 1
 2 ./the/best/it                      1
 3 ./the/best/birthdayever            1
 4 ./the/best/pizza                   1
 5 ./the/best/steaks                  1
 6 ./the/best/week                    1
 7 ./the/best/friendship              1
 8 ./the/best/an                      1
 9 ./the/best/thanks                  1
10 ./the/best/you                     1
# … with 55 more rows


nodePath,nOccurrences
<chr>,<int>
./the/best/stickum,1
./the/best/it,1
./the/best/birthdayever,1
./the/best/pizza,1
./the/best/steaks,1
./the/best/week,1
./the/best/friendship,1
./the/best/an,1
./the/best/thanks,1


In [68]:
print(rootNode, 'nOccurrences', limit = 20)

Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'self' not found”Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'super' not found”Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'private' not found”

                          levelName nOccurrences
1  start-of-sequence                       13907
2   ¦--eol                                  1000
3   ¦--the                                   374
4   ¦   ¦--day                                 7
5   ¦   ¦   ¦--u                               1
6   ¦   ¦   ¦--march                           1
7   ¦   ¦   ¦--and                             1
8   ¦   ¦   ¦--where                           1
9   ¦   ¦   ¦--plans                           1
10  ¦   ¦   ¦--hold                            1
11  ¦   ¦   °--as                              1
12  ¦   ¦--new                                 6
13  ¦   ¦   ¦--seats                           1
14  ¦   ¦   ¦--broncos                         1
15  ¦   ¦   ¦--black                           1
16  ¦   ¦   ¦--ipad                            1
17  ¦   ¦   ¦--new                             1
18  ¦   ¦   °--site                            1
19  ¦   ¦--first                               5
20  ¦   ¦   °--... 4

In [15]:
print(rootNode$'the', limit = 20)

Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'super' not found”Warning message in rm(list = names(node$children)[!keep], envir = myclone):
“object 'self' not found”

                       levelName
1  the                          
2   ¦--best                     
3   ¦   ¦--eol                  
4   ¦   ¦--in                   
5   ¦   ¦--of                   
6   ¦   ¦--kind                 
7   ¦   ¦--way                  
8   ¦   ¦--time                 
9   ¦   ¦--i                    
10  ¦   ¦--group                
11  ¦   ¦--place                
12  ¦   ¦--producers            
13  ¦   ¦--flights              
14  ¦   ¦--show                 
15  ¦   ¦--50                   
16  ¦   ¦--shows                
17  ¦   ¦--prices               
18  ¦   ¦--bits                 
19  ¦   ¦--follows              
20  ¦   °--... 48 nodes w/ 0 sub
21  °--... 1955 nodes w/ 48 sub 


In [75]:
levels <- rootNode$Get('level', 'pathString')

In [79]:
levels[2]


eol 
  2